In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Load model and tokenizer
model_name = "valhalla/t5-base-qg-hl"  # or "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize

def batch_sentences(sentences, batch_size=250):
    # Use all sentences, no max limit
    batches = [sentences[i:i+batch_size] for i in range(0, len(sentences), batch_size)]
    return batches

# Example usage:
sentences = sent_tokenize(text)
batches = batch_sentences(sentences)

for i, batch in enumerate(batches):
    print(f"Batch {i+1} length: {len(batch)}")

print(batches[0])


Batch 1 length: 250
Batch 2 length: 250
Batch 3 length: 250
Batch 4 length: 250
Batch 5 length: 250
Batch 6 length: 250
Batch 7 length: 250
Batch 8 length: 250
Batch 9 length: 250
Batch 10 length: 250
Batch 11 length: 250
Batch 12 length: 250
Batch 13 length: 250
Batch 14 length: 250
Batch 15 length: 250
Batch 16 length: 250
Batch 17 length: 250
Batch 18 length: 250
Batch 19 length: 250
Batch 20 length: 250
Batch 21 length: 250
Batch 22 length: 250
Batch 23 length: 250
Batch 24 length: 250
Batch 25 length: 250
Batch 26 length: 250
Batch 27 length: 250
Batch 28 length: 250
Batch 29 length: 250
Batch 30 length: 250
Batch 31 length: 249
['In machine learning, supervised learning (SL) is a type of machine learning paradigm where an algorithm learns to map input data to a specific output based on example input-output pairs.', 'This process involves training a statistical model using labeled data, meaning each piece of input data is provided with the correct output.', 'For instance, if you w

In [ ]:
import os
import json
import sys
import torch
from google.colab import drive

# Mount Google Drive once at start
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

batch_size = 8  # For generation batching inside each batch of 250 sentences
save_root_dir = "/content/drive/MyDrive/GPTQuestions"
os.makedirs(save_root_dir, exist_ok=True)

all_questions = []
all_answers = []

for batch_num, batchOneSentences in enumerate(batches, start=1):
    print(f"\nProcessing batch {batch_num}...")

    questions = []
    answers = []

    total_iters = len(batchOneSentences) - 2

    for start_idx in range(0, total_iters, batch_size):
        end_idx = min(start_idx + batch_size, total_iters)

        prompts = [
            "generate questions: " + " ".join(batchOneSentences[i : i+3])
            for i in range(start_idx, end_idx)
        ]
        answers.extend([" ".join(batchOneSentences[i : i+3]) for i in range(start_idx, end_idx)])

        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(device)

        outputs = model.generate(
            **inputs,
            max_length=256,
            num_beams=4,
            early_stopping=True,
            do_sample=False,
        )

        decoded = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        questions.extend(decoded)

        # Generation progress bar
        progress = end_idx / total_iters
        bar_length = 30
        filled_length = int(bar_length * progress)
        bar = '=' * filled_length + '-' * (bar_length - filled_length)
        percent = progress * 100
        sys.stdout.write(f'\rBatch {batch_num} Generation: [{bar}] {percent:.1f}% ({end_idx}/{total_iters})')
        sys.stdout.flush()

    # Add this batch's Q&A pairs to the global list
    batch_QA = [f"Question: {q} Answer: {a}" for q, a in zip(questions, answers)]
    all_questions.extend(batch_QA)

# After processing all batches, save the entire data as one file
big_file_path = os.path.join(save_root_dir, "GPTQuestions_AllBatches.json")
with open(big_file_path, "w") as f:
    json.dump(all_questions, f, indent=2)

print(f"\nAll batches processed and saved into one big file:\n{big_file_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Processing batch 1...
Batch 1 Generation: [==============================] 100.0% (248/248)
Processing batch 2...
Batch 2 Generation: [==============================] 100.0% (248/248)
Processing batch 3...
Batch 3 Generation: [==============================] 100.0% (248/248)
Processing batch 4...
Batch 4 Generation: [==============================] 100.0% (248/248)
Processing batch 5...
Batch 5 Generation: [==============================] 100.0% (248/248)
Processing batch 6...
Batch 6 Generation: [==============================] 100.0% (248/248)
Processing batch 7...
Batch 7 Generation: [==============================] 100.0% (248/248)
Processing batch 8...
Batch 8 Generation: [==============================] 100.0% (248/248)
Processing batch 9...
Batch 9 Generation: [==============================] 100.0% (248/248)
Processing batch 10...
Batch 10 Generation

In [ ]:
print(len(all_questions))
QuestionAnswer = all_questions

['Question: What is supervised learning? Answer: In machine learning, supervised learning (SL) is a type of machine learning paradigm where an algorithm learns to map input data to a specific output based on example input-output pairs. This process involves training a statistical model using labeled data, meaning each piece of input data is provided with the correct output. For instance, if you want a model to identify cats in images, supervised learning would involve feeding it many images of cats (inputs) that are explicitly labeled "cat" (outputs).', 'Question: What is the goal of supervised learning? Answer: This process involves training a statistical model using labeled data, meaning each piece of input data is provided with the correct output. For instance, if you want a model to identify cats in images, supervised learning would involve feeding it many images of cats (inputs) that are explicitly labeled "cat" (outputs). The goal of supervised learning is for the trained model t

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token by default

encodings = tokenizer(
    QuestionAnswer,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512  # or shorter if you want
)

print(type(encodings))

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [ ]:

encodings["labels"] = encodings["input_ids"].clone()

from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}



In [ ]:
train_dataset = QADataset(encodings)


In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=2,
    logging_dir="./logs",
    logging_steps=1,
    save_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: srisuresh321 (srisuresh321-california-state-university-san-marcos) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,8.070900
2,4.312700
3,2.179100
4,1.640500
5,1.399700
6,1.489100
7,0.871500
8,0.988000
9,0.994500
10,1.175800


TrainOutput(global_step=620, training_loss=0.516959262795506, metrics={'train_runtime': 186.7715, 'train_samples_per_second': 6.639, 'train_steps_per_second': 3.32, 'total_flos': 211360757760000.0, 'train_loss': 0.516959262795506, 'epoch': 5.0})

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.eval()

# Prompt text
input_text = "Please answer the following question clearly.\nQuestion: What is unsupervised learning? Answer:"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate output
outputs = model.generate(
    **inputs,
    max_length=120,           # Max length of generated sequence
    min_length=10,           # Minimum length of generated sequence
    do_sample=True,          # Enable sampling (True for randomness, False for greedy decoding)
    temperature=0.2,         # Lower is less random, higher is more random (e.g., 0.1 to 1.5)
    top_k=20,                # Limits the sampling pool to the top_k most likely next tokens
    top_p=0.9,               # Nucleus sampling, keeps tokens with cumulative prob up to top_p (0.8-0.95 common)
    repetition_penalty=1.5,  # Penalizes repeating tokens (>1 discourages repetition)
    no_repeat_ngram_size=3,  # Prevent repeating n-grams of this size (2 or 3 commonly used)
    length_penalty=1.2,      # >1 encourages longer sequences, <1 encourages shorter
    early_stopping=True,     # Stop generation when EOS token is generated
    num_return_sequences=1,  # Number of generated sequences to return
    pad_token_id=tokenizer.eos_token_id,  # Avoid warnings, set pad token id if needed
    eos_token_id=tokenizer.eos_token_id,  # End of sequence token
    decoder_start_token_id=tokenizer.pad_token_id if hasattr(tokenizer, 'pad_token_id') else None, # For some models like T5
)

# Decode and print
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Please answer the following question clearly.
Question: What is unsupervised learning? Answer: Unclassifiable data are often removed from this dataset to improve its accuracy, because of human error or other factors (e-learning). This phenomenon occurs when a large portion "is" unlabeled text – meaning that it contains irrelevant information about what's being said–and hence cannot be used as an example for future research; however in practice many researchers find ways around some aspects and even create new ones based on feedbacks such Asynchronous Processing with Disambiguation Learning via autoencoders/
